In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pydub
import os
from pydub import AudioSegment
import pandas
import numpy
import requests
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import librosa
import librosa.display
import matplotlib.pyplot as plt
import gc

In [0]:
parent_path = '/content/drive/My Drive/Experimental Projects/Bird Song/'

In [0]:
birds = pandas.read_csv(parent_path+'birds_dataset.csv', index_col=0)

In [0]:
birds.head()

,index,file_id,genus,species,name,download_url,file_name,duration,other_species_in_recording,quality
0,8,452850,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/452850/download,"XC452850-Cinereous Tinamou, song, 190117-023.mp3",53,"Poecilotriccus latirostris, Pheugopedius genib...",A
1,9,446610,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/446610/download,XC446610-Crypturellus cinereus - DaRec_2018052...,32,NaN,A
2,10,424163,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/424163/download,XC424163-Crypturellus cinereus. RO. ME- Felipe...,36,"Sclerurus obscurior, Amazona farinosa",A
3,11,333322,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/333322/download,XC333322-CRYCIN13.mp3,51,"Thamnophilus aethiops, Hypocnemis ochrogyna, M...",A
4,12,329091,Crypturellus,cinereus,Cinereous Tinamou,https://www.xeno-canto.org/329091/download,XC329091-Crypturellus cinereus(song)XapuriAC23...,37,NaN,A


In [0]:
genus_list = sorted(birds['genus'].unique().tolist())
n_classes = len(genus_list)
id_to_genus = {v: k for v, k in enumerate(genus_list)}
genus_to_id = {k: v for v, k in enumerate(genus_list)}
print('Number of unique Genera: {}'.format(n_classes))

Number of unique Genera: 182


In [0]:
import json
with open(parent_path+'id_to_genus.json', 'w') as f:
    json.dump(id_to_genus, f)
with open(parent_path+'genus_to_id.json', 'w') as f:
    json.dump(genus_to_id, f)

In [0]:
data_X = birds.download_url
data_y = birds.genus
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, stratify=data_y, test_size=0.2, random_state=100)
assert len(y_train.unique()) == n_classes
assert len(y_test.unique()) == n_classes

In [0]:
X_train_1, y_train_1 = X_train[:1796], y_train[:1796]
X_train_2, y_train_2 = X_train[1796:3592], y_train[1796:3592]
X_train_3, y_train_3 = X_train[3592:5338], y_train[3592:5338]
X_train_4, y_train_4 = X_train[5338:7184], y_train[5338:7184]
X_train_5, y_train_5 = X_train[7184:8980], y_train[7184:8980]
X_train_6, y_train_6 = X_train[8980:], y_train[8980:]

In [0]:
del X_train
del y_train
gc.collect()

0

In [0]:
def transform(x):
    x_raw = librosa.core.db_to_power(x, ref=1.0)
    x_log = numpy.log(x_raw)
    return x_log

In [0]:
def convert(src_file_name):
    dst_file_name = src_file_name + '.wav'
    sound = AudioSegment.from_mp3(src_file_name+'.mp3')[:20000]
    sound.export(dst_file_name, format="wav")
    y, sr = librosa.load(dst_file_name)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
    spect = librosa.power_to_db(spect, ref=numpy.max)
    spect = spect.T
    return spect

In [0]:
def download_convert(proc_x, proc_y, n_classes, dst_file):
    progress_counter = 0
    proc_x_accum = numpy.empty((0, 862, 128))
    proc_y_accum = numpy.empty((0, n_classes))
    err_list = []
    try:
        for index, url in proc_x.items():
            clear_output(wait=True)
            file_name = url.split('/')[3]
            file_id = int(file_name)
            file_path = parent_path+file_name
            try:    
                r = requests.get(url, allow_redirects=True)
                with open(file_path+'.mp3','wb') as op:
                    op.write(r.content)
                spect = transform(numpy.array(convert(file_path)))
                os.remove(file_path+'.mp3')
                os.remove(file_path+'.wav')
                if spect.shape[0] == 862:
                    proc_x_accum = numpy.append(proc_x_accum, [spect], axis=0)
                    # print(proc_x_accum)
                    # print(proc_x_accum.shape)
                    row_y = numpy.zeros(n_classes)
                    row_y[genus_to_id[proc_y[index]]] = 1
                    proc_y_accum = numpy.append(proc_y_accum, [row_y], axis=0)
                    progress_counter += 1
                    print('Processed {} of {} files'.format(progress_counter, proc_x.shape[0]))
            except Exception as e:
                err_list.append(str(file_id)+str(e))
    except Exception as e:
        numpy.savez(dst_file, proc_x_accum, proc_y_accum)
    numpy.savez(dst_file, proc_x_accum, proc_y_accum)

In [0]:
download_convert(X_train_1, y_train_1, n_classes, parent_path+'train_data_1')
download_convert(X_train_2, y_train_2, n_classes, parent_path+'train_data_2')
download_convert(X_train_3, y_train_3, n_classes, parent_path+'train_data_3')
download_convert(X_train_4, y_train_4, n_classes, parent_path+'train_data_4')
download_convert(X_train_5, y_train_5, n_classes, parent_path+'train_data_5')
download_convert(X_train_6, y_train_6, n_classes, parent_path+'train_data_6')

In [0]:
download_convert(X_test, y_test, n_classes, parent_path+'test_data')

Processed 2690 of 2695 files
